### Steps to load Models, make predictions and view predictions

Step 1- Run the first 12 code cells of the notebook.

Step 2- Go to the last code cell of the notebook. This cell is to load model weights. 

Step 3- Go to second last code cell of the notebook. This cell is to make predictions and load them.

Step 1 note: Don't forget to change pickle paths. This is the path your dataset is loaded from.

Step 2 note: Don't forget to modify "model_path" variable so you can use the shared model weight files. This is the path where your model weights reside in.

Step 3 Note: Don't forget to modify "plot_path" variable. This variable determines where to save the plot.

### Links of the shared data and weight files

__________________________

Links of the shared Weight Files:

250 epochs:

	https://drive.google.com/drive/folders/1xUVIkAGRHxXUhNj-V2-q6M7pnXSn_V4e?usp=sharing


500 epochs:

	https://drive.google.com/drive/folders/1nPecL3pfFkPDTQTnz3PofWWtdk-u3IZF?usp=sharing

______________________

Links of the shared Pickle Files:

	Half Faces:
  
		Ground Truths:

			https://drive.google.com/file/d/1-JVnG_wVJR3VgAwi6-Hhu2C-ZAyQ2-_9/view?usp=sharing

		Occluded Images:
			https://drive.google.com/file/d/1-7E0x-UGFjotUH8UJAWruM9Y0rwEzYzV/view?usp=sharing



	Unrestricted Occlusions:

		Ground Truths:

			https://drive.google.com/file/d/19li26wV60jhrf8UtUhGH6xuocDqiHqPG/view?usp=sharing

		Occluded Images:

			https://drive.google.com/file/d/179YgtbT7A0YFJsQyFULbQPgPZzdmnySA/view?usp=sharing


### The Code

Step 1- Run the first 12 code cells


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

import keras
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dropout, LeakyReLU, Conv2DTranspose, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Reshape
from tensorflow.keras import layers
import datetime

from keras import initializers


config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # fraction of memory
config.gpu_options.visible_device_list = "0"

set_session(tf.Session(config=config))

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Step 1 note: Don't forget to change pickle paths.

In [0]:

path = '/content/drive/My Drive/biometrics_data/'
xname = '1_half_face_occluded.pickle'
yname = '1_half_face_labels.pickle'
pickle_in = open(os.path.join(path,xname),"rb")
x = pickle.load(pickle_in)

pickle_in = open(os.path.join(path,yname),"rb")
y = pickle.load(pickle_in)

print(x.shape)
print(y.shape)

In [0]:
#I use this because with 200,200 images, I exceed GPU memory.

from skimage.transform import resize
x = resize(x, (len(x),64,64,1), anti_aliasing=False)
y = resize(y, (len(y),64,64,1), anti_aliasing=False)

# Print the shape after resize
print(x.shape)
print(y.shape)

#Draw the image to be sure occluded image is the same as the ground truth one
fig=plt.figure(figsize=(6, 6))
fig.add_subplot(1, 2, 1)
plt.imshow(x[1,:,:,0],cmap="gray")
fig.add_subplot(1, 2, 2)
plt.imshow(y[1,:,:,0],cmap="gray")

In [0]:
#I print pixel values to be sure occluded and ground truth images are aligned
print(x[0,0:5,0:5,0])
print(y[0,0:5,0:5,0])

In [0]:
#Here is the function to create a generator. I also use the function and create a generator in the same cell.
def creategen():
  generator = Sequential()


  generator.add(Conv2D(64, (5,5) , strides = (2,2), input_shape = x.shape[1:] , padding = "SAME",kernel_initializer = 'random_normal'))
  generator.add(BatchNormalization())
  generator.add(ReLU())  
  generator.add(Dropout(0.3))


  generator.add(Conv2D(128, (5,5) ,  strides = (2,2),padding = "SAME",kernel_initializer = 'random_normal'))
  generator.add(BatchNormalization())
  generator.add(ReLU())  
  generator.add(Dropout(0.3))

  
  generator.add(Conv2D(256, (5,5) ,  strides = (2,2), padding = "SAME",kernel_initializer = 'random_normal'))
  generator.add(BatchNormalization())
  generator.add(ReLU())  
  generator.add(Dropout(0.3))

  # I would use these if there was a bottleneck in the network.
  #generator.add(Flatten())
  
  #generator.add(Dense(64)) 
  #generator.add(BatchNormalization())
  #generator.add(ReLU(alpha=0.2))
  
  #generator.add(Dense(8*8*128)) 
  #generator.add(BatchNormalization())
  #generator.add(ReLU(alpha=0.2))
  
  #generator.add(layers.Reshape((8,8,128)))


  generator.add(Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
  generator.add(BatchNormalization())
  generator.add(ReLU())
  
  generator.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
  generator.add(BatchNormalization())
  generator.add(ReLU())
  
  generator.add(Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation = "tanh"))
  

  return generator

generator = creategen()



In [0]:
#Here is the function to create a discriminator. 
#I also use the function and create a discriminator in the same cell.
def createdisc():
  discriminator = Sequential()

  discriminator.add(Conv2D(64, (5,5) , strides = (2,2), input_shape = x.shape[1:] , padding = "SAME",kernel_initializer = 'random_normal'))
  discriminator.add(BatchNormalization())
  discriminator.add(LeakyReLU(alpha=0.2))  
  discriminator.add(Dropout(0.3))


  discriminator.add(Conv2D(128, (5,5) ,  strides = (2,2),padding = "SAME",kernel_initializer = 'random_normal'))
  discriminator.add(BatchNormalization())
  discriminator.add(LeakyReLU(alpha=0.2))  
  discriminator.add(Dropout(0.3))

  
  discriminator.add(Conv2D(256, (5,5) ,  strides = (2,2), padding = "SAME",kernel_initializer = 'random_normal'))
  discriminator.add(BatchNormalization())
  discriminator.add(LeakyReLU(alpha=0.2))  
  discriminator.add(Dropout(0.3))

  
  discriminator.add(Flatten())
  discriminator.add(Dense(1))
  
  return discriminator


discriminator = createdisc()



In [0]:
generator.summary()

In [0]:
# Learning rate of discriminator is advised to be the double of the generator. 
# I train the generator two times in each step of training. First is with Gan loss, second is with L2 loss.
# That is why I use x4 learning rate in discriminator.

opt_disc = Adam(lr=0.00004)
discriminator.trainable = True
discriminator.compile(loss = "binary_crossentropy", optimizer = opt_disc)
discriminator.summary()

In [0]:
opt_gen = Adam(lr=0.00001)
generator.compile(loss='mean_squared_error', optimizer = opt_gen)
generator.summary()

In [0]:
#Here is the function to create a GAN model. I also use the function and create GAN model in the same cell.

def creategan(generator,discriminator):
  gan = Sequential()
  gan.add(generator)
  discriminator.trainable = False
  gan.add(discriminator)
  return(gan)

gan = creategan(generator,discriminator)

In [0]:
opt_gan = Adam(lr=0.00001)
gan.compile(loss = "binary_crossentropy", optimizer = opt_gan)
gan.summary()

In [0]:
# This is to save models. It saves both the architectures and the weights.
# While loading the model, you will only need weights since you declared the architecture in the previous cells.

def save_models(gan,discriminator,generator,path,epoch):
  datenow = str(datetime.datetime.now().strftime('%m-%d-%H:%M'))
  gan.save_weights(os.path.join(path,"{0}_wgan_{1}.h5".format(epoch,datenow)))
  gan.save(os.path.join(path,"{0}_mgan_{1}.h5".format(epoch,datenow)))


  discriminator.save_weights(os.path.join(path,"{0}_wd_{1}.h5".format(epoch,datenow)))
  discriminator.save(os.path.join(path,"{0}_md_{1}.h5".format(epoch,datenow)))


  generator.save_weights(os.path.join(path,"{0}_wg_{1}.h5".format(epoch,datenow)))
  generator.save(os.path.join(path,"{0}_mg_{1}.h5".format(epoch,datenow)))
  

  


In [0]:
# This is to obtain test losses in each training step. I use this function in the "train" function.
def test(x,y):
  gan_inp_t = x  
  gan_label_t = np.ones([len(gan_inp_t)])
  gan_predict_t = None
  
  disc_inp_t = None


  disc_label_t = np.zeros([len(gan_inp_t)*2])
  disc_label_t[len(gan_inp_t):] = 1
  
  disc_predict_t = None
  

  gen_predict_t = generator.predict(gan_inp_t)


  disc_inp_t = np.concatenate((gen_predict_t,y), axis = 0)
  disc_predict_t = discriminator.predict(disc_inp_t)

  d_loss_t = discriminator.test_on_batch(disc_inp_t,disc_label_t)

  gan_loss_t = gan.test_on_batch(gan_inp_t,gan_label_t)
  
  return (gan_loss_t, d_loss_t)

In [0]:
# In the previous versions, I used this function to pretrain the discriminator in each training step.
# I do not use this function in this version.
def pretrain(x,y,nepoch):
    
  gen_predict = None
  
  
  gan_inp = x  
  gan_label = np.ones(64)
  gan_predict = None
  
  disc_inp = None


  disc_label = np.zeros(64*2)
  disc_label[64:] = 1
  
  disc_predict = None
  
  
  for epoch in range(nepoch):
    for batch_ctr in range(65):


      gen_predict = generator.predict(gan_inp[batch_ctr*64:(batch_ctr+1)*64])


      disc_inp = np.concatenate((gen_predict,y[batch_ctr*64:(batch_ctr+1)*64]), axis = 0)
      disc_predict = discriminator.predict(disc_inp)



      d_loss = discriminator.train_on_batch(disc_inp,disc_label)


      gan_loss = gan.test_on_batch(gan_inp[batch_ctr*64:(batch_ctr+1)*64],
                                    gan_label)
    

    (tgan,tdisc)= test(x[4160:],y[4160:])
    print("Pretrain Epoch Gan Loss: {0}       Disc Loss: {1}".format(gan_loss,d_loss))
    print("Pretrain Epoch Test Gan Loss: {0}  Test Disc Loss: {1} \n\n\n".format(tgan,tdisc))

In [0]:
# I use isolated mini batches to slow down discriminator:
# I also use smooth/noisy labels proposed by Salimans et al 2016
# Reference: https://github.com/soumith/ganhacks



def train(x,y, nepoch, model_save_path = "/content/drive/My Drive/biometrics_data/models/dummymodel"):

  gen_predict = None
  
  #Initialize the inputs and the labels.
  gan_inp = x  
  gan_label = np.ones(64)
  gan_predict = None
  
  disc_inp = None


  disc_label = np.zeros(64*2)
  disc_label[64:] = 1
  
  disc_predict = None
  

  sess = tf.Session()
  

  for epoch in range(nepoch):

    for batch_ctr in range(65):

      #Generator makes a prediction.
      gen_predict = generator.predict(gan_inp[batch_ctr*64:(batch_ctr+1)*64])


      #Minibatch isolation and label smoothing is done here:
      
      if(epoch%2==0):
        disc_inp = gen_predict
        #disc_label = np.zeros(64) //// I would use this line if there was no noisy labels.
        disc_label = np.random.normal(loc=0, scale=0.10, size=64)
      else:
        disc_inp = y[batch_ctr*64:(batch_ctr+1)*64]
        #disc_label = np.ones(64)  //// I would use this line if there was no noisy labels.
        disc_label = np.random.normal(loc=1, scale=0.10, size=64)


        
      #Initialize a label variable for generator to use it in training.  
      gen_label = y[batch_ctr*64:(batch_ctr+1)*64]
      
      
      #Do one training step. Also assign the losses to variables. We will print them.
      d_loss = discriminator.train_on_batch(disc_inp,disc_label)


      gan_loss = gan.train_on_batch(gan_inp[batch_ctr*64:(batch_ctr+1)*64],
                                    gan_label)

      gen_loss = generator.train_on_batch(gan_inp[batch_ctr*64:(batch_ctr+1)*64],gen_label)
      
    
    
    if epoch+1%500 == 0:
      save_models(gan,discriminator,generator, model_save_path,epoch+1)  
      print("MODEL SAVED")
      
    
    # Test images are the images after the 4160th image. It makes 311 test images.
    (tgan,tdisc)= test(x[4160:],y[4160:])
    print("Epoch: {2} Gan Loss: {0}       Disc Loss: {1}        Gen Loss: {3}".format(gan_loss,d_loss,epoch+1,gen_loss))
    print("Epoch: {2} Test Gan Loss: {0}  Test Disc Loss: {1} \n\n\n".format(tgan,tdisc,epoch+1))

In [0]:
# Checking lengths of the input and ground truth arrays. Also checking if normalization is done.
print(len(x),len(y))
print(x.max(),x.min())
print(y.max(),y.min())


If you would like to make trainings, don't forget to modify "model_save_path" variable.

In [0]:
# Use this cell if you have loaded and viewed the results, and want to train further.
# If you start the model from beginning, using above cells where I declare 
# generator architecture, discriminator architecture etc. , you don't need to use this cell since
# I already compile the models in above cells.


# Learning rate of discriminator is advised to be the double of the generator. 
# I train the generator two times in each step of training. First is with Gan loss, second is with L2 loss.
# That is why I use x4 learning rate in discriminator.

"""opt_disc = Adam(lr=0.00004)
discriminator.trainable = True
discriminator.compile(loss = "binary_crossentropy", optimizer = opt_disc)
discriminator.summary()

opt_gen = Adam(lr=0.00001)
generator.compile(loss='mean_squared_error', optimizer = opt_gen)
generator.summary()

opt_gan = Adam(lr=0.00001)
gan.compile(loss = "binary_crossentropy", optimizer = opt_gan)
gan.summary()"""

In [0]:
# To train, don't forget to:
# -compile the models
# - run the necessary cells to declare the necessary functions such as "train" and "save_models" and "test"

#If you train models from beginning, just run the cells of the notebook in order.

#If you want to further train loaded models, use the previous cell to compile models.

model_save_path = "/content/drive/My Drive/biometrics_data/models/half_face"
batch_size = 64
train(x,y,504,model_save_path)

Step 3 - Make predictions and view them.

Step 3 Note: Don't forget to modify "plot_path" variable. This variable determines where to save the plot.

In [0]:
#Making predictions and drawing them.
#First row: Occluded images
#Second row: Ground Truth images
#Third row: Predictions

import datetime
plot_path = "/content/drive/My Drive/biometrics_data/plots/half_face"

a = 4160
b = 4170
pred=generator.predict(x[a:b])

fig = plt.figure(figsize = (20,10))
for ctr in range(10):
  fig.add_subplot(3,10,ctr+1)
  plt.imshow(np.reshape(x[a + ctr],(64,64)),  cmap = "gray")

  

for ctr in range(10):
  fig.add_subplot(3,10,(10 + ctr + 1))
  plt.imshow(np.reshape(y[a + ctr]/255,(64,64)),  cmap = "gray")  


for ctr in range(10):
  fig.add_subplot(3,10,(20 + ctr + 1))
  plt.imshow(np.reshape(pred[ctr],(64,64)),  cmap = "gray")
  
plt.savefig(os.path.join(plot_path,str(datetime.datetime.now().strftime('%m-%d-%H:%M'))))

Step 2- Load weights here:
(Don't forget to modify the model_path)

In [0]:
model_path = "/content/drive/My Drive/biometrics_data/models/last_experiment_250/"

epoch = 250



generator = creategen()
generator.load_weights(os.path.join(model_path,"{0}_wg.h5".format(epoch)))
discriminator = createdisc()
discriminator.load_weights(os.path.join(model_path,"{0}_wd.h5".format(epoch)))


gan = creategan(generator,discriminator)
gan.load_weights(os.path.join(model_path,"{0}_wgan.h5".format(epoch)))